In [1]:
# Copyright 2024 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Engine in Vertex AI


### Agent Engine in Vertex AI

[Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) (LangChain on Vertex AI) is a managed service that helps you to build and deploy an agent reasoning framework. It gives you the flexibility to choose how much reasoning you want to delegate to the LLM and how much you want to handle with customized code. You can define Python functions that get used as tools via [Gemini Function Calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling). Reasoning Engine integrates closely with the Python SDK for the Gemini model in Vertex AI, and it can manage prompts, agents, and examples in a modular way. Reasoning Engine is compatible with LangChain, LlamaIndex, or other Python frameworks.


### Install Vertex AI SDK for Python

In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[agent_engines,langchain,ag2]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 65.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
# Note:  Need to restart the kernel
%pip install --upgrade --user --quiet langchain-google-vertexai

In [4]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
#MODEL_NAME = "gemini-1.5-flash-002"
MODEL_NAME = "gemini-2.0-flash-001"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initial set up

In [5]:
# @title Create a bucket.
BUCKET_URI = f"gs://agent-0319"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0319/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0319' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [6]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [7]:
# @title Set access to the bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://agent-0319/
No changes made to gs://agent-0319/


In [8]:
# @title Initialize Vertex AI with Staging Bucket.

import vertexai
import langchain_google_vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Tool

In [9]:
# @title Tool define
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [10]:
# Tool test
get_exchange_rate(currency_from="USD", currency_to="KRW")

{'amount': 1.0, 'base': 'USD', 'date': '2025-03-18', 'rates': {'KRW': 1453.26}}

## Local Agent

In [11]:

from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

model_kwargs = {

    "temperature": 0.28,
    "max_output_tokens": 1000,
    "top_p": 0.95,
    "top_k": None,
    "safety_settings": safety_settings,
}

In [12]:
# @title Agent Define - Local unit test
from vertexai.preview.reasoning_engines import LangchainAgent

local_agent = LangchainAgent(
    model = MODEL_NAME,
    model_kwargs=model_kwargs,
    tools = [get_exchange_rate],

    # agent_executor_kwargs={"return_intermediate_steps": False},
)

In [13]:
# @title Test your agent locally

# Agent local unit test before deploying.
local_agent.query(input="What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025/03/10 ?")

{'input': "What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025/03/10 ?",
 'output': 'OK. The exchange rate from USD to KRW on 2025-03-10 is 1455.89. That means 1 USD equals 1455.89 KRW.\n'}

## Remote Agent

In [14]:
# @title Deploy your agent on Vertex AI

from vertexai import agent_engines

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = agent_engines.create(
    local_agent,
    display_name="currency agent",
    gcs_dir_name = "ai-agent-1",
    description="This is a simple reasoning engine.",
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain,ag2]",
        "langchain-google-vertexai",
        "cloudpickle==3.0",
    ],
    extra_packages = []
)

INFO:vertexai.agent_engines:Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.85.0'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,langchain,ag2]', 'langchain-google-vertexai', 'cloudpickle==3.0']
INFO:vertexai.agent_engines:Using bucket agent-0319
INFO:vertexai.agent_engines:Wrote to gs://agent-0319/ai-agent-1/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://agent-0319/ai-agent-1/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://agent-0319/ai-agent-1/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/8505338167358914560/operations/3434805159018889216
INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=ai-

In [15]:
# @title Query from remote engine.
remote_agent.query(
    input="What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025/03/10 ?"
)

{'input': "What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025/03/10 ?",
 'output': 'OK. The exchange rate from USD to KRW on 2025-03-10 is 1455.89. That means 1 USD equals 1455.89 KRW.\n'}

## Agent Management

In [17]:
# @title Helper function to manage reasoning engine.

# Properties of ReasoningEngine class.
# https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/manage/overview

#----------------------------------------
def get_agent_engine(display_name:str):
  try:
    for agent in agent_engines.list():
      if agent.display_name == display_name:
        return agent_engines.get(agent.name)

      else:
        print("No such reasoning engine or Invalid display name.")

  except Exception as e:
    print(e)

#----------------------------------------
def show_agents():
  """
  List reasoning engines.
  """

  try:
    if not agent_engines.list():
      print("No reasoning engines")

    for idx, agent in enumerate(agent_engines.list()):
        print(f"Agent {idx}: \n\tDisplay Name [{agent.display_name}] \n\tName [{agent.name}] \n\tCreation Time [{agent.create_time}] \n\tResource Name [{agent.resource_name}]\n")

  except Exception as e:
    print(e)

#----------------------------------------

def delete_agent(name):
  """
  Delete a reasoning engines.
  @param name: The name of the reasoning engine.
  @type name: str
  """

  try:
    re = agent_engines.get(name)
    re.delete()
    print(f"Deleted {name}")
  except Exception as e:
    print(e)


In [18]:
show_agents()

Agent 0: 
	Display Name [currency agent] 
	Name [8505338167358914560] 
	Creation Time [2025-03-19 02:48:56.849333+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/8505338167358914560]

Agent 1: 
	Display Name [simple agent 2] 
	Name [8470716745223503872] 
	Creation Time [2025-03-19 00:56:04.818945+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/8470716745223503872]



In [19]:
get_agent_engine(display_name="currency agent")


resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/8505338167358914560

In [20]:
#delete_agent(name="4754121152736002048")

In [21]:
# @title Get instances with display name.
remote_agent2 = get_agent_engine('currency agent')

remote_agent2.query(
    input="What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025/03/10 ?"
)

{'input': "What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025/03/10 ?",
 'output': 'OK. The exchange rate from USD to KRW on 2025-03-10 is 1455.89. That means 1 USD equals 1455.89 KRW.\n'}